In [ ]:
import os
import torch
import json
from tqdm.auto import tqdm
from internal.renderers.gsplat_renderer import GSPlatRenderer
from internal.renderers.gsplat_hit_pixel_count_renderer import GSplatHitPixelCountRenderer
from internal.dataparsers.colmap_dataparser import Colmap, ColmapDataParser
from internal.utils.sh_utils import RGB2SH
from internal.utils.gaussian_model_loader import GaussianModelLoader

In [ ]:
torch.autograd.set_grad_enabled(False)

In [ ]:
partition_base_dir = os.path.expanduser(
    "~/data/image_set/JNUCar_undistorted/colmap/drone/dense_max_2048/0/partitions-threshold_0.2/")

In [ ]:
dataparser_outputs = ColmapDataParser(
    os.path.join(partition_base_dir, ".."),
    output_path=os.getcwd(),
    global_rank=0,
    params=Colmap(
        appearance_groups="appearance_image_dedicated",
        eval_step=32
    ),
).get_outputs()

In [ ]:
image_name_to_camera_idx = {}
for idx, name in enumerate(dataparser_outputs.train_set.image_names):
    image_name_to_camera_idx[name] = idx
image_name_to_camera_idx

In [ ]:
def get_cameras_by_image_list(image_list: list):
    cameras = []
    for i in image_list:
        cameras.append(dataparser_outputs.train_set.cameras[image_name_to_camera_idx[i]])
    return cameras

In [ ]:
partitions = torch.load(os.path.join(partition_base_dir, "partitions.pt"),
                        map_location="cpu")

orientation_transformation = partitions["orientation_transformation"]

model_paths = []
for idx, i in enumerate(partitions["ids"]):
    if len(partitions["image_indices"][idx]) < 32:
        continue
    model_paths.append((i, os.path.join(
        "../outputs/JNUAerial-0526/",
        f"P_{i[0]:03d}_{i[1]:03d}.txt")))
partitions.keys(), model_paths

In [ ]:
partition_id_to_index = {i: idx for idx, i in enumerate(partitions["ids"])}
partition_id_to_index

In [ ]:
# group name to image list
with open(os.path.join(partition_base_dir, "..", "appearance_image_dedicated.json"), "r") as f:
    appearance_groups = json.load(f)
# image name to group name
image_name_to_group_name = {}
for group_name in appearance_groups:
    image_names = appearance_groups[group_name]
    for image_name in image_names:
        image_name_to_group_name[image_name] = group_name

In [ ]:
# def calculate_partition_features_dc(model_output_path, partition_id, ckpt, is_in_partition):
#     # the ids of the groups in this model
#     appearance_group_ids = torch.load(os.path.join(model_output_path, "appearance_group_ids.pth"))
# 
#     # load partition image list, then get their correspond appearance ids
#     partition_used_appearance_ids = []
#     with open(os.path.join(partition_base_dir, f"{partition_id[0]:03d}_{partition_id[1]:03d}.txt"), "r") as f:
#         for row in f:
#             image_group_name = image_name_to_group_name[row.rstrip("\n")]
#             partition_used_appearance_ids.append(appearance_group_ids[image_group_name][0])
# 
#     # get average appearance embeddings of these used images
#     average_appearance_embedding = ckpt["hyper_parameters"]["renderer"].model.embedding(
#         torch.tensor(partition_used_appearance_ids, dtype=torch.int)
#     ).mean(dim=0)
# 
#     # calculate rgb_offset from appearance embedding
#     partition_features_extra = ckpt["state_dict"]["gaussian_model._features_extra"][is_in_partition]
#     appearance_mlp_input = torch.concat([partition_features_extra, average_appearance_embedding.unsqueeze(0).repeat(
#         partition_features_extra.shape[0], 1)], dim=-1)
#     rgb_offset = ckpt["hyper_parameters"]["renderer"].model.network.to("cuda")(appearance_mlp_input.to("cuda")) * 2 - 1.
# 
#     # calculate base_rgb from features_dc
#     base_rgb = spherical_harmonics(0, torch.ones(partition_features_extra.shape[0], 3).to("cuda"),
#                                    ckpt["state_dict"]["gaussian_model._features_dc"][is_in_partition].to("cuda")) + 0.5
# 
#     # calculate final rgb values
#     rgbs = (base_rgb + rgb_offset).clamp(0., 1.)
# 
#     # convert to SHs
#     partition_features_dc = RGB2SH(rgbs).unsqueeze(1)
# 
#     return partition_features_dc

In [ ]:
def calculate_gaussian_scores(cameras, gaussian_model, device):
    hit_count_list = []
    opacity_score_list = []
    alpha_score_list = []
    all_visibility_score = torch.zeros((len(cameras), gaussian_model.get_xyz.shape[0]), dtype=torch.float, device=device)
    for idx, camera in tqdm(enumerate(cameras), total=len(cameras)):
        hit_count, opacity_score, alpha_score, visibility_score = GSplatHitPixelCountRenderer.hit_pixel_count(
            means3D=gaussian_model.get_xyz,
            opacities=gaussian_model.get_opacity,
            scales=gaussian_model.get_scaling,
            rotations=gaussian_model.get_rotation,
            viewpoint_camera=camera.to_device("cuda"),
        )
        # hit_count_list.append(hit_count.cpu())
        # opacity_score_list.append(opacity_score.cpu())
        # alpha_score_list.append(alpha_score.cpu())
        all_visibility_score[idx] = visibility_score.to(device=device)
        # visibility_score_list.append(visibility_score.cpu())

    torch.cuda.empty_cache()

    return all_visibility_score

In [ ]:
def prune_gaussian_model(gaussian_model, mask):
    gaussian_model.properties = {k: v[mask] for k, v in gaussian_model.properties.items()}

In [ ]:
def get_pruned_gaussian_model(image_list, ckpt, is_in_partition):
    cuda_device = torch.device("cuda")
    
    # get camera correspond to the image list
    cameras = get_cameras_by_image_list(image_list)
    # build appearance id, the order is the same as the list `cameras`
    camera_index_to_appearance_id = torch.tensor([i.appearance_id for i in cameras], dtype=torch.int, device=cuda_device)

    gaussian_model = GaussianModelLoader.initialize_model_from_checkpoint(
        ckpt,
        "cpu",
    )
    # prune those outside bounding box
    prune_gaussian_model(gaussian_model, is_in_partition)
    gaussian_model.to(device=cuda_device)
    
    # get renderer
    renderer = GaussianModelLoader.initialize_renderer_from_checkpoint(ckpt, stage="validation", device=cuda_device)

    # calculate Gaussians' visibility score to each camera; the output may consume a lot of memory, so put it on CPU
    visibility_score = calculate_gaussian_scores(cameras, gaussian_model, "cpu").T  # [N_gaussians, N_cameras]
    # calculate total visibility score for each Gaussian
    visibility_score_acc = torch.sum(visibility_score, dim=-1)
    # find Gaussian whose total visibility is closed to zero
    visibility_score_acc_is_close_to_zero = torch.isclose(visibility_score_acc, torch.tensor(0., device=visibility_score_acc.device))
    gaussian_to_preserve = ~visibility_score_acc_is_close_to_zero
    # prune again
    prune_gaussian_model(gaussian_model, gaussian_to_preserve.to(device=cuda_device))
    visibility_score_pruned = visibility_score[~visibility_score_acc_is_close_to_zero]
    del visibility_score
    
    """
    [NOTE]
    The codes below is related to `GSplatAppearanceEmbeddingRenderer`, aiming to remove the dependency on MLP.
    If you have not enabled this model, simply comment out them.
    
    === `GSplatAppearanceEmbeddingRenderer` related code start ===
    """

    # get top `n_average_cameras` visibility cameras
    n_average_cameras = 32
    visibility_score_pruned_sorted = torch.topk(visibility_score_pruned, k=n_average_cameras, dim=-1)
    visibility_score_pruned_sorted_values = visibility_score_pruned_sorted.values.to(device=cuda_device)
    visibility_score_pruned_sorted_indices = visibility_score_pruned_sorted.indices.to(device=cuda_device)
    del visibility_score_pruned_sorted
    
    visibility_score_pruned_top_k_acc = torch.sum(visibility_score_pruned_sorted_values, dim=-1, keepdim=True)
    # calculate the weight of each camera
    visibility_score_pruned_top_k_pdf = visibility_score_pruned_sorted_values / visibility_score_pruned_top_k_acc
    assert torch.all(torch.isclose(visibility_score_pruned_top_k_pdf.sum(dim=-1), torch.tensor(1., device=visibility_score_pruned_top_k_pdf.device)))

    # pick appearance id
    appearance_ids = camera_index_to_appearance_id[visibility_score_pruned_sorted_indices.reshape(-1)]  # [N_gaussians * n_average_cameras]
    # pick appearance embeddings
    appearance_embeddings = renderer.model.embedding(appearance_ids).reshape((
        visibility_score_pruned_sorted_indices.shape[0],
        n_average_cameras,
        -1,
    ))  # [N_gaussians, n_average_cameras, N_embedding_dims]
    # multiply embeddings by camera weights
    weighted_appearance_embeddings = appearance_embeddings * visibility_score_pruned_top_k_pdf.unsqueeze(-1)
    # merge `n_average_cameras` embedding to a single embedding
    final_appearance_embeddings = torch.sum(weighted_appearance_embeddings, dim=1)

    # embedding network forward, output rgb_offset
    embedding_network = renderer.model.network
    input_tensor_list = [
        gaussian_model.get_appearance_features(),
        final_appearance_embeddings,
    ]
    
    # view dependent
    if renderer.model_config.is_view_dependent:
        camera_index_to_camera_center = torch.stack(
            [i.camera_center for i in cameras], 
        ).to(device=cuda_device)  # [N_cameras, 3]
        camera_centers = camera_index_to_camera_center[visibility_score_pruned_sorted_indices]  # [N_gaussians, n_average_cameras, 3]
        
        # not sure which one below is better
        
        # [OPTION 1] weighted camera centers
        # weighted_camera_centers = camera_centers * visibility_score_pruned_top_k_pdf.unsqueeze(-1)
        # final_camera_centers = torch.sum(weighted_camera_centers, dim=1)  # [N_gaussians, 3]
        # view_directions = torch.nn.functional.normalize(gaussian_model.get_means() - final_camera_centers, dim=-1)
        
        # [OPTION 2] weighted view directions
        unweighted_view_directions = torch.nn.functional.normalize(gaussian_model.get_means().unsqueeze(1) - camera_centers, dim=-1)  # [N_gaussians, n_average_cameras, 3]
        weighted_view_directions = unweighted_view_directions * visibility_score_pruned_top_k_pdf.unsqueeze(-1)
        view_directions = torch.nn.functional.normalize(torch.sum(weighted_view_directions, dim=1), dim=-1)  # [N_gaussians, 3]
        
        encoded_view_directions = renderer.model.view_direction_encoding(view_directions)
        
        input_tensor_list.append(encoded_view_directions)
    
    input_tensor = torch.concat(input_tensor_list, dim=-1).to(cuda_device)
    rgb_offset = embedding_network(input_tensor)
    # convert rgb_offset to SHs
    sh_offset = RGB2SH(rgb_offset)

    gaussian_model.shs_dc = gaussian_model.shs_dc + sh_offset.unsqueeze(1)
    
    """ 
    === `GSplatAppearanceEmbeddingRenderer` related code end ===
    """
    
    gaussian_model.to(device)

    return gaussian_model

In [ ]:
dtype = torch.float
device = torch.device("cpu")

n_sh_degrees = 0
xyzs = torch.zeros((0, 3), dtype=dtype, device=device)
features_dc = torch.zeros((0, 1, 3), dtype=dtype, device=device)
features_rest = torch.zeros((0, ((n_sh_degrees + 1) ** 2 - 1), 3), dtype=dtype, device=device)
scales = torch.zeros((0, 3), dtype=dtype, device=device)
rotations = torch.zeros((0, 4), dtype=dtype, device=device)
opacities = torch.zeros((0, 1), dtype=dtype, device=device)

with tqdm(model_paths) as t:
    for i in t:
        # if i[0] != (0, 0):
        #     continue
        with open(os.path.join(i[1], "cameras.json"), "r") as f:
            cameras_json = json.load(f)
        image_list = [i["img_name"] for i in cameras_json]

        partition_xy = partitions["xys"][partition_id_to_index[i[0]]]
        load_file = GaussianModelLoader.search_load_file(i[1])
        t.set_description(f"{partition_xy}: {load_file}")
        ckpt = torch.load(load_file, map_location=device)
        xyz = ckpt["state_dict"]["gaussian_model.gaussians.means"]
        reoriented_xyz = xyz @ orientation_transformation[:3, :3].T
        # include min bound, exclude max bound
        is_in_partition = torch.logical_and(torch.ge(reoriented_xyz[:, :2], partition_xy),
                                            torch.lt(reoriented_xyz[:, :2], partition_xy + 2 * partitions["radius"]))
        is_in_partition = torch.logical_and(is_in_partition[:, 0], is_in_partition[:, 1])

        pruned_gaussian_model = get_pruned_gaussian_model(image_list, ckpt, is_in_partition)

        xyzs = torch.concat([xyzs, pruned_gaussian_model.means])
        features_dc = torch.concat([features_dc, pruned_gaussian_model.shs_dc])
        features_rest = torch.concat([features_rest, pruned_gaussian_model.shs_rest])
        scales = torch.concat([scales, pruned_gaussian_model.scales])
        rotations = torch.concat([rotations, pruned_gaussian_model.rotations])
        opacities = torch.concat([opacities, pruned_gaussian_model.opacities])

        torch.cuda.empty_cache()

In [ ]:
# model_output_path = i[1]
# model_output_path

In [ ]:
# # load partition image list, then get their correspond appearance ids
# partition_id = i[0]
# partition_used_appearance_ids = []
# image_list = []
# with open(os.path.join(partition_base_dir, f"{partition_id[0]:03d}_{partition_id[1]:03d}.txt"), "r") as f:
#     for row in f:
#         image_list.append(row.rstrip("\n"))
# 
# cameras = get_cameras_by_image_list(image_list)
# state_dict = {}
# for i in ckpt["state_dict"]:
#     if i.startswith("gaussian_model._"):
#         state_dict[i] = ckpt["state_dict"][i][is_in_partition]
# gaussian_model = GaussianModelSimplified.construct_from_state_dict(
#     state_dict,
#     active_sh_degree=ckpt["hyper_parameters"]["gaussian"].sh_degree,
#     device="cuda",
# )

In [ ]:
# for i in state_dict:
#     ckpt["state_dict"][i] = state_dict[i]
# torch.save(ckpt, os.path.join(model_output_path, "checkpoints", "pruned.ckpt"))

In [ ]:
import gc

gc.collect()

In [ ]:
# # get the contribution to every camera of each Gaussian
# visibility_score = calculate_gaussian_scores(cameras, gaussian_model)  # [N_cameras, N_gaussians]

In [ ]:
# visibility_score = visibility_score.T

In [ ]:
# visibility_score.shape  # [N_gaussians, N_cameras]

In [ ]:
# visibility_score_acc = torch.sum(visibility_score, dim=-1)

In [ ]:
# visibility_score_acc_is_close_to_zero = torch.isclose(visibility_score_acc, torch.tensor(0.))
# visibility_score_acc_is_close_to_zero.sum()

In [ ]:
# visibility_score_acc_is_close_to_zero.shape

In [ ]:
# for i in state_dict:
#     state_dict[i] = state_dict[i][~visibility_score_acc_is_close_to_zero]
# for i in state_dict:
#     ckpt["state_dict"][i] = state_dict[i]
# torch.save(ckpt, os.path.join(model_output_path, "checkpoints", "pruned-closed-to-zero.ckpt"))

In [ ]:
# visibility_score_pruned = visibility_score[~visibility_score_acc_is_close_to_zero]

In [ ]:
# # get top `n_average_cameras` camera by contribution
# n_average_cameras = 32
# visibility_score_pruned_sorted = torch.topk(visibility_score_pruned, k=n_average_cameras, dim=-1)

In [ ]:
# visibility_score_pruned_sorted.indices.shape

In [ ]:
# visibility_score_pruned_sorted.indices[0], visibility_score_pruned_sorted.values[0]

In [ ]:
# if some visibility of the 2nd-Kst cameras are closed to zero, use the 1st camera
# visibility_score_sorted.indices[:, 1:] = torch.where(
#     torch.isclose(visibility_score_sorted.values[:, 1:], torch.tensor(0.)),
#     visibility_score_sorted.indices[:, :1],
#     visibility_score_sorted.indices[:, 1:],
# )
# visibility_score_sorted.indices[0], visibility_score_sorted.values[0]

In [ ]:
# visibility_score_pruned_top_k_acc = torch.sum(visibility_score_pruned_sorted.values, dim=-1, keepdim=True)
# visibility_score_pruned_top_k_acc.shape

In [ ]:
# calculate the weight of each camera
# visibility_score_pruned_top_k_pdf = visibility_score_pruned_sorted.values / visibility_score_pruned_top_k_acc
# visibility_score_pruned_top_k_pdf.shape, torch.all(
#     torch.isclose(visibility_score_pruned_top_k_pdf.sum(dim=-1), torch.tensor(1.)))

In [ ]:
# camera_index_to_appearance_id = torch.tensor([i.appearance_id for i in cameras], dtype=torch.int)
# camera_index_to_appearance_id

In [ ]:
# appearance_ids = camera_index_to_appearance_id[visibility_score_pruned_sorted.indices.reshape(-1)]
# appearance_ids.shape

In [ ]:
# appearance_embeddings = ckpt["hyper_parameters"]["renderer"].model.embedding(appearance_ids).reshape(
#     (visibility_score_pruned_sorted.indices.shape[0], n_average_cameras, -1))

In [ ]:
# weighted_appearance_embeddings = appearance_embeddings * visibility_score_pruned_top_k_pdf.unsqueeze(-1)

In [ ]:
# final_appearance_embeddings = torch.sum(weighted_appearance_embeddings, dim=1)
# final_appearance_embeddings.shape

In [ ]:
# embedding_network = ckpt["hyper_parameters"]["renderer"].model.network.to("cuda")

In [ ]:
# input_tensor = torch.concat([state_dict["gaussian_model._features_extra"], final_appearance_embeddings], dim=-1).to(
#     "cuda")

In [ ]:
# rgb_offset = embedding_network(input_tensor)
# rgb_offset

In [ ]:
# sh_offset = RGB2SH(rgb_offset)

In [ ]:
# state_dict["gaussian_model._features_dc"].shape, sh_offset.unsqueeze(1).cpu().shape

In [ ]:
# new_features_dc = state_dict["gaussian_model._features_dc"] + sh_offset.unsqueeze(1).cpu()

In [ ]:
# state_dict["gaussian_model._features_dc"] = new_features_dc

In [ ]:
# remove `GSplatAppearanceEmbeddingRenderer`'s states from ckpt
state_dict_key_to_delete = []
for i in ckpt["state_dict"]:
    if i.startswith("renderer."):
        state_dict_key_to_delete.append(i)
for i in state_dict_key_to_delete:
    del ckpt["state_dict"][i]

In [ ]:
ckpt["optimizer_states"] = []

In [ ]:
# ckpt["hyper_parameters"]["renderer"] = GSPlatRenderer()

In [ ]:
# for i in state_dict:
#     ckpt["state_dict"][i] = state_dict[i]
# torch.save(ckpt, os.path.join(model_output_path, "checkpoints", "pruned-gsplat-vanilla-renderer.ckpt"))

In [ ]:
# replace `AppearanceFeatureGaussian` with `VanillaGaussian`
from internal.models.vanilla_gaussian import VanillaGaussian
ckpt["hyper_parameters"]["gaussian"] = VanillaGaussian(sh_degree=pruned_gaussian_model.max_sh_degree)

In [ ]:
# remove existing Gaussians from ckpt
for i in list(ckpt["state_dict"].keys()):
    if i.startswith("gaussian_model.gaussians."):
        del ckpt["state_dict"][i]

In [ ]:
ckpt["state_dict"]["gaussian_model.gaussians.means"] = xyzs
ckpt["state_dict"]["gaussian_model.gaussians.shs_dc"] = features_dc
ckpt["state_dict"]["gaussian_model.gaussians.shs_rest"] = features_rest
ckpt["state_dict"]["gaussian_model.gaussians.scales"] = scales
ckpt["state_dict"]["gaussian_model.gaussians.rotations"] = rotations
ckpt["state_dict"]["gaussian_model.gaussians.opacities"] = opacities
ckpt["hyper_parameters"]["renderer"] = GSPlatRenderer()  # replace `GSplatAppearanceEmbeddingRenderer` with `GSPlatRenderer`

In [ ]:
torch.save(ckpt, "jnu_aerial-0526.ckpt")

Merging is completed here

# [NOTE] Contents below are optional or just for experimental purposes

Update a specific partition

In [ ]:
dtype = torch.float
device = torch.device("cpu")

input_path = "../edited/20240415_232032.ckpt"
ckpt = torch.load(input_path, map_location=device)

In [ ]:
xyz = ckpt["state_dict"]["gaussian_model._xyz"]
reoriented_xyz = xyz @ orientation_transformation[:3, :3].T

In [ ]:
target_partition_id = (0, 0)
target_partition_ckpt = "../outputs/JNUAerial/P_000_000.txt-random_background/checkpoints/epoch=19-step=30000.ckpt"

In [ ]:
partition_xy = partitions["xys"][partition_id_to_index[target_partition_id]]
partition_xy

In [ ]:
# include min bound, exclude max bound
is_in_partition = torch.logical_and(torch.ge(reoriented_xyz[:, :2], partition_xy),
                                    torch.lt(reoriented_xyz[:, :2], partition_xy + 2 * partitions["radius"]))
is_in_partition = torch.logical_and(is_in_partition[:, 0], is_in_partition[:, 1])
is_in_partition.sum()

In [ ]:
# remove point inside partition
not_in_partition = ~is_in_partition
xyzs = ckpt["state_dict"]["gaussian_model._xyz"][not_in_partition]
features_dc = ckpt["state_dict"]["gaussian_model._features_dc"][not_in_partition]
features_rest = ckpt["state_dict"]["gaussian_model._features_rest"][not_in_partition]
scales = ckpt["state_dict"]["gaussian_model._scaling"][not_in_partition]
rotations = ckpt["state_dict"]["gaussian_model._rotation"][not_in_partition]
opacities = ckpt["state_dict"]["gaussian_model._opacity"][not_in_partition]
xyzs.shape

In [ ]:
partition_ckpt = torch.load(target_partition_ckpt, map_location=device)
reoriented_partition_xyz = partition_ckpt["state_dict"]["gaussian_model._xyz"] @ orientation_transformation[:3, :3].T

In [ ]:
is_in_partition = torch.logical_and(torch.ge(reoriented_partition_xyz[:, :2], partition_xy),
                                    torch.lt(reoriented_partition_xyz[:, :2], partition_xy + 2 * partitions["radius"]))
is_in_partition = torch.logical_and(is_in_partition[:, 0], is_in_partition[:, 1])
is_in_partition.sum()

In [ ]:
xyzs = torch.concat([xyzs, partition_ckpt["state_dict"]["gaussian_model._xyz"][is_in_partition]])
features_dc = torch.concat([features_dc, partition_ckpt["state_dict"]["gaussian_model._features_dc"][is_in_partition]])
features_rest = torch.concat(
    [features_rest, partition_ckpt["state_dict"]["gaussian_model._features_rest"][is_in_partition]])
scales = torch.concat([scales, partition_ckpt["state_dict"]["gaussian_model._scaling"][is_in_partition]])
rotations = torch.concat([rotations, partition_ckpt["state_dict"]["gaussian_model._rotation"][is_in_partition]])
opacities = torch.concat([opacities, partition_ckpt["state_dict"]["gaussian_model._opacity"][is_in_partition]])

In [ ]:
ckpt["state_dict"]["gaussian_model._xyz"] = xyzs
ckpt["state_dict"]["gaussian_model._features_dc"] = features_dc
ckpt["state_dict"]["gaussian_model._features_rest"] = features_rest
ckpt["state_dict"]["gaussian_model._scaling"] = scales
ckpt["state_dict"]["gaussian_model._rotation"] = rotations
ckpt["state_dict"]["gaussian_model._opacity"] = opacities

In [ ]:
torch.save(ckpt, "jnu_aerial_new.ckpt")

LightGaussian

In [ ]:
from internal.utils.light_gaussian import get_count_and_score, calculate_v_imp_score, get_prune_mask

In [ ]:
# get partition image list
partition_id = (0, 0)
image_list = []
with open(os.path.join(partition_base_dir, f"{partition_id[0]:03d}_{partition_id[1]:03d}.txt"), "r") as f:
    for row in f:
        image_list.append(row.rstrip("\n"))

# get camera correspond to the image list
cameras = get_cameras_by_image_list(image_list)

In [ ]:
ckpt = torch.load("../outputs/JNUAerial-0526/P_000_000.txt/checkpoints/epoch=100-step=132700.ckpt", map_location="cpu")

In [ ]:
partition_xy = partitions["xys"][partition_id_to_index[partition_id]]
partition_xy

In [ ]:
xyz = ckpt["state_dict"]["gaussian_model._xyz"]
reoriented_xyz = xyz @ orientation_transformation[:3, :3].T
# include min bound, exclude max bound
is_in_partition = torch.logical_and(torch.ge(reoriented_xyz[:, :2], partition_xy),
                                    torch.lt(reoriented_xyz[:, :2], partition_xy + 2 * partitions["radius"]))
is_in_partition = torch.logical_and(is_in_partition[:, 0], is_in_partition[:, 1])

In [ ]:
# get Gaussians located in partition to construct a new state_dict
state_dict = {}
for i in ckpt["state_dict"]:
    if i.startswith("gaussian_model._"):
        state_dict[i] = ckpt["state_dict"][i][is_in_partition]

In [ ]:
gaussian_model = GaussianModelSimplified.construct_from_state_dict(state_dict, 0, device="cuda")

In [ ]:
hit_camera_count_total = torch.zeros((gaussian_model.get_xyz.shape[0]), dtype=torch.int,
                                     device=gaussian_model.get_xyz.device)
opacity_score_total = torch.zeros((gaussian_model.get_xyz.shape[0]), dtype=torch.float,
                                  device=gaussian_model.get_xyz.device)
alpha_score_total = torch.zeros((gaussian_model.get_xyz.shape[0]), dtype=torch.float,
                                device=gaussian_model.get_xyz.device)
visibility_score_total = torch.zeros((gaussian_model.get_xyz.shape[0]), dtype=torch.float,
                                     device=gaussian_model.get_xyz.device)

In [ ]:
for idx, camera in tqdm(enumerate(cameras), total=len(cameras)):
    hit_count, opacity_score, alpha_score, visibility_score = GSplatHitPixelCountRenderer.hit_pixel_count(
        means3D=gaussian_model.get_xyz,
        opacities=gaussian_model.get_opacity,
        scales=gaussian_model.get_scaling,
        rotations=gaussian_model.get_rotation,
        viewpoint_camera=camera.to_device("cuda"),
    )
    hit_camera_count_total += hit_count
    opacity_score_total += opacity_score
    alpha_score_total += alpha_score
    visibility_score_total += visibility_score

In [ ]:
visibility_close_to_zero = torch.isclose(visibility_score_total, torch.tensor(0.).to(visibility_score_total))
visibility_close_to_zero.sum(), visibility_score_total.shape[0]

In [ ]:
# gaussian_model.delete_gaussians(visibility_close_to_zero)

In [ ]:
# visible_gaussians = torch.bitwise_not(visibility_close_to_zero)
# hit_camera_count_total = hit_camera_count_total[visible_gaussians]
# opacity_score_total = opacity_score_total[visible_gaussians]
# alpha_score_total = alpha_score_total[visible_gaussians]
# visibility_score_total = visibility_score_total[visible_gaussians]

In [ ]:
# (visibility_score_total > 1).sum(), visibility_score_total.shape[0]

In [ ]:
visibility_score_total_sorted = torch.sort(visibility_score_total, descending=True)
visibility_score_total_sorted

In [ ]:
preserve_indices = visibility_score_total_sorted.indices[:int(visibility_score_total_sorted.indices.shape[0] * 0.1)]
preserve_indices.shape[0]

In [ ]:
for i in state_dict:
    ckpt["state_dict"][i] = state_dict[i][preserve_indices.to(ckpt["state_dict"][i].device)].to(
        ckpt["state_dict"][i].device)

In [ ]:
torch.save(ckpt, "visibility_pruned.ckpt")

v_imp_score

In [ ]:
v_imp_score = calculate_v_imp_score(gaussian_model.get_scaling, opacity_score_total, 0.1)
v_imp_score, v_imp_score.shape

In [ ]:
prune_mask = get_prune_mask(0.9, v_imp_score)
preserve_mask = torch.bitwise_not(prune_mask)
preserve_mask.sum(), preserve_mask.shape

In [ ]:
for i in state_dict:
    ckpt["state_dict"][i] = state_dict[i][preserve_mask.to(ckpt["state_dict"][i].device)].to(
        ckpt["state_dict"][i].device)

In [ ]:
torch.save(ckpt, "opacity_score_pruned.ckpt")